# ALP Interface

We will automate the ALP through its COM Interface 

In [1]:
from src import alp
import os
from pathlib import Path
from tqdm import tqdm
import os
import pandas as pd
from src.pile import PipePile
import numpy as np

DEBUG:matplotlib:matplotlib data path: c:\Users\dazhong.li\AppData\Local\miniconda3\envs\UnitCPT\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\dazhong.li\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\dazhong.li\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\dazhong.li\.matplotlib\fontlist-v330.json


We will use SetNodePY function

In [2]:
model = alp.ALP()
model.new_file('test.alw','','UnitCPT','Push-Over Analysis','CPT p-y analysis')
# model.open(r'C:\Users\dazhong.li\OneDrive - Arup\01 - Project\UnitCPT\UnitCPT\alpmodel.alw')
# model.save()

proj = model.proj
pile_dia =2.0 # 1.0m
pile_thickness = 0.05 # 50mm
pile_length = 80 # 
pile_penetration = 80# 
corrosion = 0

test_pile = PipePile(pile_dia,pile_thickness,pile_length,pile_penetration,corrosion=0)
model.set_section(1,f'Tubular Pile {pile_dia}m, thickness={pile_thickness:.3f}',1,1,test_pile.EI)

# Need to set to Specified_PY
proj.SetSoilModel(1)

DEBUG:root:Terminated - alp.exe Pid-13360
DEBUG:root:Started New file test.alw


In [3]:
# df = pd.read_excel('./p-y curves cyclic - CPT04.xlsx',sheet_name='SUMMARY-Static',skiprows=2)

In [3]:
# df = pd.read_excel('cpt_data_resampled.xlsx')

def calc_displacement_curve(df,name):
    model = alp.ALP()
    model.new_file('test.alw','','UnitCPT','Push-Over Analysis','CPT p-y analysis')
    # model.open(r'C:\Users\dazhong.li\OneDrive - Arup\01 - Project\UnitCPT\UnitCPT\alpmodel.alw')
    # model.save()
    proj = model.proj
    pile_dia =2.0 # 1.0m
    pile_thickness = 0.05 # 50mm
    pile_length = 80 # 
    pile_penetration = 80# 
    corrosion = 0

    test_pile = PipePile(pile_dia,pile_thickness,pile_length,pile_penetration,corrosion=0)
    model.set_section(1,f'Tubular Pile {pile_dia}m, thickness={pile_thickness:.3f}',1,1,test_pile.EI)

    # Need to set to Specified_PY
    proj.SetSoilModel(1)
    column_str =['p0','y0','p2','y2','p4','y4','p6','y6','p8','y8','p12','y12']
    df_alp = df[['Depth','p0','y0','p2','y2','p4','y4','p6','y6','p8','y8','p12','y12']].copy()
    df_alp['Level'] = df_alp.Depth*-1
    df_alp= df_alp.dropna()
    # proj.ClearNodes()
    for i, row in df_alp.iterrows():
        proj.InsertNode(row['Level'],1)
        proj.AddNodePY(*(row[column_str].values.tolist()))
    df_load_disp_curve = pd.DataFrame()
    force = []
    disp = []
    rotation = []
    i = 0
    for f in tqdm(np.linspace(0,5000,21)):
        model.set_node_load_displacement(1,f*-1,0,0)
        model.analyse()
        df_load_disp_curve.loc[i,'H0'] = f
        df_load_disp_curve.loc[i,'u0'] = model.proj.GetNodeDisp(1)
        df_load_disp_curve.loc[i,'theta0'] = model.proj.GetNodeRotation(1)
        i +=1
    i = 0
    for m in tqdm(np.linspace(0,5000,21)):
        model.set_node_load_displacement(1,0,m,0)
        model.analyse()
        df_load_disp_curve.loc[i,'M0'] = m
        df_load_disp_curve.loc[i,'u1'] = model.proj.GetNodeDisp(1)
        df_load_disp_curve.loc[i,'theta1'] = model.proj.GetNodeRotation(1)
        i +=1
    model.save_as(name)
    return df_load_disp_curve

In [4]:
case = 'SUMMARY-Static'
df = pd.read_excel('../p-y curves cyclic - CPT04.xlsx',sheet_name=case,skiprows=2)
df.drop([0,1],axis=0,inplace=True)
df_load_disp_curve = calc_displacement_curve(df,r'C:\Users\dazhong.li\OneDrive - Arup\01 - Project\UnitCPT\UnitCPT\ALP Calibration'+f'\{case}.alw')
df_load_disp_curve.to_json(f'{case}.json')

FileNotFoundError: [Errno 2] No such file or directory: '../p-y curves cyclic - CPT04.xlsx'

In [38]:
df_load_disp_static =pd.read_json('SUMMARY-Static.json')
df_load_disp_cyclic =pd.read_json('SUMMARY-Cyclic.json')

In [39]:
from src import geoplot
import  plotly.graph_objects as go
markers = geoplot.GEOPlot.marker_list()

def plot_load_disp(df_load_disp_curve, name, symbol, fig=None):
    if fig is None:
        fig = geoplot.GEOPlot.get_figure(rows=2,cols=2)
    fig.add_trace(go.Scatter(y=df_load_disp_curve.H0,x=df_load_disp_curve.u0,
                                mode='markers+lines',
                                line=dict(color='black'),
                                marker=dict(symbol=symbol),
                                name=name,
                                showlegend=True),row=1,col=1)
    fig.add_trace(go.Scatter(y=df_load_disp_curve.H0,x=df_load_disp_curve.theta0,
                                mode='markers+lines',
                                line=dict(color='black'),
                                marker=dict(symbol=symbol),
                                showlegend=False),row=1,col=2)
    fig.update_xaxes(title='Pile Head Displacement (mm)',row=1,col=1)
    fig.update_xaxes(title='Pile Head Rotation (rad)',row=1,col=2)
    fig.update_yaxes(title='Applied Lateral Force(H=H0, M=0) at Pile Head(kN)',row=1,col=1)

    fig.add_trace(go.Scatter(y=df_load_disp_curve.M0,x=df_load_disp_curve.u1,
                            mode='markers+lines',
                            line=dict(color='black'),
                            marker=dict(symbol=symbol),
                            showlegend=False),row=2,col=2)
    fig.add_trace(go.Scatter(y=df_load_disp_curve.M0,x=df_load_disp_curve.theta1,
                            mode='markers+lines',
                            marker=dict(symbol=symbol),
                            line=dict(color='black'),showlegend=False),
                            row=2,col=1)
    fig.update_xaxes(title='Pile Head Displacement (mm)',row=2,col=2)
    fig.update_xaxes(title='Pile Head Rotation (rad)',row=2,col=1)
    fig.update_yaxes(title='Applied Lateral Force at Pile Head(kN)',row=2,col=1)
    fig.update_layout(width=800,height=600,font=dict(size=8))
    return fig


In [40]:
fig = plot_load_disp(df_load_disp_static,'Static','circle')
plot_load_disp(df_load_disp_cyclic,'cyclic','circle-open',fig=fig)

# Estimate Stiffness matrices using ALP

$$
\begin{equation}
\begin{bmatrix} H\\ M\\ \end{bmatrix} = \overbrace{\begin{bmatrix} K_L & K_{LR} \\ k_{LR} & K_R \end{bmatrix}}^{\text{Stiffness Matrix}} \begin{bmatrix} \rho\\ \theta \end{bmatrix}
\Rightarrow
\begin{bmatrix} \rho\\ \theta\\ \end{bmatrix} = \begin{bmatrix} K_L & K_{LR} \\ k_{LR} & K_R \end{bmatrix}^{-1} \begin{bmatrix} H \\ M \end{bmatrix}
\Rightarrow
\begin{bmatrix} \rho\\ \theta\\ \end{bmatrix} = \begin{bmatrix} I_L & I_{LR} \\ I_{LR} & I_R \end{bmatrix} \begin{bmatrix} H \\ M \end{bmatrix}
\end{equation}
$$

To obtain this stiffness matrix, it is easier to run the analysis in two runs:

    H = H0 and M=0

    H = 0 and M=M0

$$
\begin{equation}
\begin{bmatrix} \rho_0\\ \theta_0\\ \end{bmatrix} = \begin{bmatrix} I_L & I_{LR} \\ I_{LR} & I_R \end{bmatrix} \begin{bmatrix} H \\ 0 \end{bmatrix} \Rightarrow
\begin{bmatrix}
\rho_0 = H_0 \cdot I_L \\ \theta_0 = H_0 \cdot I_{LR} \end{bmatrix} \Rightarrow
\begin{bmatrix} I_L = \frac{\rho_0}{H_0} \\ I_{LR} = \frac{\theta_0}{H_0} \end{bmatrix} 
\end{equation}
$$


$$
\begin{equation}
\begin{bmatrix} \rho_1\\ \theta_1\\ \end{bmatrix} = \begin{bmatrix} I_L & I_{LR} \\ I_{LR} & I_R \end{bmatrix} \begin{bmatrix} 0 \\ M_0 \end{bmatrix} \Rightarrow
\begin{bmatrix}
\rho_1 = M_0 \cdot I_{LR} \\ \theta_1 = M_0 \cdot I_{R} \end{bmatrix} \Rightarrow
\begin{bmatrix} I_{LR} = \frac{\rho_1}{M_0} \\ I_{R} = \frac{\theta_1}{M_0} \end{bmatrix} 
\end{equation}
$$




In [66]:
from scipy.interpolate import interp1d

H0 = 300
M0 = 300

f_u0 = interp1d(df_load_disp_curve.H0,df_load_disp_curve.u0)
f_u1 = interp1d(df_load_disp_curve.M0,df_load_disp_curve.u1)
f_theta0 = interp1d(df_load_disp_curve.H0,df_load_disp_curve.theta0)
f_theta1 = interp1d(df_load_disp_curve.M0,df_load_disp_curve.theta1)

u0 = f_u0(H0)
u1 = f_u1(M0)
rho0 = f_rho0(H0)
rho1 = f_rho1(M0)

I_matrix = np.array([[u0/H0,rho0/H0],[u1/M0,rho1/M0]])